In [374]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from shapely.geometry import Point
df = gpd.read_file('/data/acker/ALA/NA/final/NAv5_GWRPM25_mean_2020-2022.shp')
#df = gpd.read_file('/data/acker/ALA/NA/NAv5_GWRPM25_mean2016-2018_99th.shp')
df

,lat,lon,GWRPM25,geometry
0,14.005,-91.455,28.300001,POINT (-91.455 14.005)
1,14.005,-91.445,28.333334,POINT (-91.445 14.005)
2,14.005,-91.435,28.066666,POINT (-91.435 14.005)
3,14.005,-91.425,28.333334,POINT (-91.425 14.005)
4,14.005,-91.415,28.266668,POINT (-91.415 14.005)
...,...,...,...,...
22526035,67.995,-107.485,1.933333,POINT (-107.485 67.995)
22526036,67.995,-107.315,2.100000,POINT (-107.315 67.995)
22526037,67.995,-107.305,2.100000,POINT (-107.305 67.995)
22526038,67.995,-107.295,2.100000,POINT (-107.295 67.995)


In [375]:
#upload shapefile of U.S. counties
counties = gpd.read_file('/data/acker/shapefiles/cb_2020_us_county_500k.shp')

In [376]:
# List of state abbreviations for CONUS, Alaska (AK), and Hawaii (HI)
states_to_include = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'DC',
    'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS',
    'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
    'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV',
    'WI', 'WY'
]

# Filter counties to only include rows where STUSPS is in the specified list
counties_conus = counties[counties['STUSPS'].isin(states_to_include)]

# Display the filtered counties
counties_conus

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,13,031,00350302,0500000US13031,13031,Bulloch,Bulloch County,GA,Georgia,06,1750735216,32827779,"POLYGON ((-82.02684 32.55516, -82.02527 32.558..."
1,13,121,01694833,0500000US13121,13121,Fulton,Fulton County,GA,Georgia,06,1364133162,19666288,"POLYGON ((-84.84931 33.51318, -84.84429 33.514..."
2,13,179,00357095,0500000US13179,13179,Liberty,Liberty County,GA,Georgia,06,1337654102,222884791,"POLYGON ((-81.8244 32.01488, -81.81338 32.0162..."
3,13,189,00348794,0500000US13189,13189,McDuffie,McDuffie County,GA,Georgia,06,666590014,23114032,"POLYGON ((-82.64852 33.60838, -82.64409 33.606..."
4,13,213,01688005,0500000US13213,13213,Murray,Murray County,GA,Georgia,06,892235145,5719644,"POLYGON ((-84.94434 34.68004, -84.9431 34.6804..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,41,049,01135860,0500000US41049,41049,Morrow,Morrow County,OR,Oregon,06,5259046087,44372730,"POLYGON ((-120.00631 45.27648, -120.00556 45.3..."
3229,21,073,00516883,0500000US21073,21073,Franklin,Franklin County,KY,Kentucky,06,538312209,10786432,"POLYGON ((-85.02426 38.13059, -85.02299 38.131..."
3230,17,201,01785216,0500000US17201,17201,Winnebago,Winnebago County,IL,Illinois,06,1328792556,15873993,"POLYGON ((-89.40142 42.50043, -89.37175 42.500..."
3231,17,123,00424260,0500000US17123,17123,Marshall,Marshall County,IL,Illinois,06,1001843226,30364348,"POLYGON ((-89.63869 40.98691, -89.63868 40.988..."


In [377]:
counties_conus = counties_conus.drop(["COUNTYNS", 'NAMELSAD', 'LSAD', 'ALAND', 'AWATER', 'AFFGEOID'], axis=1)

In [378]:
counties_conus = counties_conus.to_crs(df.crs)

In [379]:
import geopandas as gpd
import pandas as pd

# Parameters
chunk_size = 10000  # Adjust this based on your memory capacity

# Initialize an empty list to store results
results = []

# Iterate over chunks of the data
for i in range(0, len(df), chunk_size):
    chunk = df.iloc[i:i+chunk_size]
    chunk_result = gpd.sjoin(chunk, counties_conus[['GEOID', 'geometry','STATEFP','COUNTYFP']], how="left", predicate="within")
    chunk_result.dropna(subset=['GEOID'], inplace=True)  # Drop rows where GEOID is NaN in this chunk
    results.append(chunk_result)

# Concatenate all chunk results
df_counties = pd.concat(results, ignore_index=True)


In [380]:
df_counties

,lat,lon,GWRPM25,geometry,index_right,GEOID,STATEFP,COUNTYFP
0,18.915,-155.675,2.800000,POINT (-155.675 18.915),269.0,15001,15,001
1,18.925,-155.675,2.700000,POINT (-155.675 18.925),269.0,15001,15,001
2,18.925,-155.665,2.566667,POINT (-155.665 18.925),269.0,15001,15,001
3,18.925,-155.655,2.533334,POINT (-155.655 18.925),269.0,15001,15,001
4,18.935,-155.675,2.700000,POINT (-155.675 18.935),269.0,15001,15,001
...,...,...,...,...,...,...,...,...
10125518,67.995,-141.045,3.000000,POINT (-141.045 67.995),2780.0,02290,02,290
10125519,67.995,-141.035,3.000000,POINT (-141.035 67.995),2780.0,02290,02,290
10125520,67.995,-141.025,3.033334,POINT (-141.025 67.995),2780.0,02290,02,290
10125521,67.995,-141.015,3.066667,POINT (-141.015 67.995),2780.0,02290,02,290


In [381]:
large = df_counties[df_counties['GWRPM25'] >= 22]
large

,lat,lon,GWRPM25,geometry,index_right,GEOID,STATEFP,COUNTYFP
2471663,36.195,-118.765,22.500000,POINT (-118.765 36.195),431.0,06107,06,107
2471664,36.195,-118.755,22.699999,POINT (-118.755 36.195),431.0,06107,06,107
2471665,36.195,-118.745,22.666666,POINT (-118.745 36.195),431.0,06107,06,107
2476199,36.205,-118.785,22.500000,POINT (-118.785 36.205),431.0,06107,06,107
2476200,36.205,-118.775,22.633333,POINT (-118.775 36.205),431.0,06107,06,107
...,...,...,...,...,...,...,...,...
6145084,43.885,-122.235,22.900000,POINT (-122.235 43.885),1047.0,41039,41,039
6145085,43.885,-122.225,23.200001,POINT (-122.225 43.885),1047.0,41039,41,039
6145086,43.885,-122.215,23.650000,POINT (-122.215 43.885),1047.0,41039,41,039
6145087,43.885,-122.205,24.099998,POINT (-122.205 43.885),1047.0,41039,41,039


In [382]:
large.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [383]:
'''import geopandas as gpd
import matplotlib.pyplot as plt
from geopandas.tools import overlay

# Load U.S. map shapefile (example uses 'naturalearth_lowres')
us_map = counties_conus

# Load your GeoDataFrame (replace 'your_geodataframe' with your actual data)
gdf = large  # Replace with your loaded GeoDataFrame

# Ensure CRS matches (assuming both are in EPSG:4326)
gdf = gdf.set_crs(epsg=4269)
us_map = us_map.to_crs(epsg=4269)

# Plot the data
fig, ax = plt.subplots(figsize=(10, 8))
us_map.plot(ax=ax, color='lightgray', edgecolor='black')  # Plot U.S. map
gdf.plot(ax=ax, color='red', markersize=5)  # Plot points from GeoDataFrame

# Add titles and labels
plt.title('NA PM2.5 > EPA Max (22 ug/m^3)', fontsize=16)
plt.xlabel('Longitude', fontsize=12)
plt.ylabel('Latitude', fontsize=12)
plt.show()
'''

"import geopandas as gpd\nimport matplotlib.pyplot as plt\nfrom geopandas.tools import overlay\n\n# Load U.S. map shapefile (example uses 'naturalearth_lowres')\nus_map = counties_conus\n\n# Load your GeoDataFrame (replace 'your_geodataframe' with your actual data)\ngdf = large  # Replace with your loaded GeoDataFrame\n\n# Ensure CRS matches (assuming both are in EPSG:4326)\ngdf = gdf.set_crs(epsg=4269)\nus_map = us_map.to_crs(epsg=4269)\n\n# Plot the data\nfig, ax = plt.subplots(figsize=(10, 8))\nus_map.plot(ax=ax, color='lightgray', edgecolor='black')  # Plot U.S. map\ngdf.plot(ax=ax, color='red', markersize=5)  # Plot points from GeoDataFrame\n\n# Add titles and labels\nplt.title('NA PM2.5 > EPA Max (22 ug/m^3)', fontsize=16)\nplt.xlabel('Longitude', fontsize=12)\nplt.ylabel('Latitude', fontsize=12)\nplt.show()\n"

In [427]:
# Group by GEOID and calculate the 90th percentile of GWRPM25 for each county
#county_90th_percentile = df_counties.groupby('GEOID')['GWRPM25'].quantile(0.9).reset_index()
#county_90th_percentile = df_counties.groupby('GEOID')['GWRPM25'].max().reset_index()
county_90th_percentile = df_counties.groupby('GEOID')['GWRPM25'].mean().reset_index()


# Rename the column for clarity
county_90th_percentile = county_90th_percentile.rename(columns={'GWRPM25': 'PM25_90th'})

# Display the result
print(county_90th_percentile)


      GEOID  PM25_90th
0     01001   7.480436
1     01003   8.249587
2     01005   7.732464
3     01007   7.691876
4     01009   7.403364
...     ...        ...
3137  56037   6.010496
3138  56039   3.983185
3139  56041   6.375806
3140  56043   6.194958
3141  56045   8.896798

[3142 rows x 2 columns]


In [428]:
# Merge the 90th percentile data with the counties GeoDataFrame to include geometry
county_90th_percentile_with_geometry = county_90th_percentile.merge(counties[['GEOID', 'geometry','STATEFP','COUNTYFP']], on='GEOID', how='left')

# Convert to a GeoDataFrame if necessary
county_90th_percentile_with_geometry = gpd.GeoDataFrame(county_90th_percentile_with_geometry, geometry='geometry')

# Display the result
print(county_90th_percentile_with_geometry)


      GEOID  PM25_90th                                           geometry  \
0     01001   7.480436  POLYGON ((-86.9212 32.65754, -86.92035 32.6585...   
1     01003   8.249587  POLYGON ((-88.02858 30.22676, -88.02399 30.230...   
2     01005   7.732464  POLYGON ((-85.74803 31.61918, -85.74544 31.618...   
3     01007   7.691876  POLYGON ((-87.42194 33.00338, -87.33177 33.005...   
4     01009   7.403364  POLYGON ((-86.96336 33.85822, -86.95967 33.857...   
...     ...        ...                                                ...   
3137  56037   6.010496  POLYGON ((-110.05438 42.01103, -110.05436 42.0...   
3138  56039   3.983185  POLYGON ((-111.05361 44.66627, -110.75076 44.6...   
3139  56041   6.375806  POLYGON ((-111.04663 41.14012, -111.04662 41.1...   
3140  56043   6.194958  POLYGON ((-108.55063 44.15179, -108.55056 44.1...   
3141  56045   8.896798  POLYGON ((-105.08123 43.65128, -105.0808 43.71...   

     STATEFP COUNTYFP  
0         01      001  
1         01      003  
2  

In [429]:
# epa CDVs for pearson
gdf_epa = pd.read_csv('/data/acker/EPA_DV/2020-2022.csv')

In [430]:
gdf_epa

,State Name,County Name,State FIPS,County FIPS,Design Value
0,Alabama,Baldwin,1,3,7.5
1,Alabama,Clay,1,27,6.8
2,Alabama,DeKalb,1,49,7.2
3,Alabama,Etowah,1,55,8.2
4,Alabama,Jefferson,1,73,8.4
...,...,...,...,...,...
518,Wyoming,Laramie,56,21,4.9
519,Wyoming,Natrona,56,25,3.8
520,Wyoming,Park,56,29,4.9
521,Wyoming,Sublette,56,35,3.6


In [431]:
#gdf_epa.rename(columns={'2016-2018': 'Design Value'}, inplace=True)

In [432]:

# Function to add leading zero to 11-digit codes
def add_leading_zero(code):
    code_str = str(code).strip()
    if len(code_str) == 1:
        ret = '0' + code_str
        return ret
    return code_str

In [433]:
# Iterate through each block group ID in ejscreen and update if necessary
for i in range(len(gdf_epa)):
    gdf_epa.at[i, 'State FIPS'] = add_leading_zero(gdf_epa.at[i, 'State FIPS'])

# Iterate through each block group ID in ejscreen and update if necessary
for i in range(len(gdf_epa)):
    gdf_epa.at[i, 'County FIPS'] = add_leading_zero(gdf_epa.at[i, 'County FIPS'])


/tmp/ipykernel_310756/958764295.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gdf_epa.at[i, 'State FIPS'] = add_leading_zero(gdf_epa.at[i, 'State FIPS'])
/tmp/ipykernel_310756/958764295.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '03' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gdf_epa.at[i, 'County FIPS'] = add_leading_zero(gdf_epa.at[i, 'County FIPS'])


In [434]:

# Function to add leading zero to 11-digit codes
def add_leading_zero(code):
    code_str = str(code).strip()
    if len(code_str) == 2:
        ret = '0' + code_str
        return ret
    return code_str


for i in range(len(gdf_epa)):
    gdf_epa.at[i, 'County FIPS'] = add_leading_zero(gdf_epa.at[i, 'County FIPS'])


In [435]:
county_90th_percentile_with_geometry.columns

Index(['GEOID', 'PM25_90th', 'geometry', 'STATEFP', 'COUNTYFP'], dtype='object')

In [436]:
import numpy as np

# Set values that are 0 in 'Design Value' to NaN in gdf_epa
gdf_epa.loc[gdf_epa['Design Value'] == 0, 'Design Value'] = np.nan

# Drop rows where 'Design Value' is NaN, so only counties with valid EPA data remain
gdf_epa.dropna(subset=['Design Value'], inplace=True)

# Step 1: Create a unique identifier combining 'STUSPS' and 'NAME' for both datasets
gdf_epa['ID'] = gdf_epa['State FIPS'] + "_" + gdf_epa['County FIPS']
county_90th_percentile_with_geometry['ID'] = county_90th_percentile_with_geometry['STATEFP'] + "_" + county_90th_percentile_with_geometry['COUNTYFP']

# Step 2: Identify IDs with valid values in gdf_epa
epa_ids = gdf_epa['ID'].unique()

# Step 3: Identify unmonitored counties
unmonitored_counties = county_90th_percentile_with_geometry[
    ~county_90th_percentile_with_geometry['ID'].isin(epa_ids)
]

# Step 4: Update PM25_90th in county_90th_percentile_with_geometry to NaN for IDs not in epa_ids
county_90th_percentile_with_geometry.loc[
    ~county_90th_percentile_with_geometry['ID'].isin(epa_ids), 'PM25_90th'
] = np.nan


In [437]:
len(epa_ids)

523

In [438]:
unmonitored_counties

,GEOID,PM25_90th,geometry,STATEFP,COUNTYFP,ID
0,01001,7.480436,"POLYGON ((-86.9212 32.65754, -86.92035 32.6585...",01,001,01_001
2,01005,7.732464,"POLYGON ((-85.74803 31.61918, -85.74544 31.618...",01,005,01_005
3,01007,7.691876,"POLYGON ((-87.42194 33.00338, -87.33177 33.005...",01,007,01_007
4,01009,7.403364,"POLYGON ((-86.96336 33.85822, -86.95967 33.857...",01,009,01_009
5,01011,7.624880,"POLYGON ((-85.99926 32.25018, -85.98655 32.250...",01,011,01_011
...,...,...,...,...,...,...
3135,56033,5.701011,"POLYGON ((-107.91152 45.00067, -107.90818 45.0...",56,033,56_033
3137,56037,6.010496,"POLYGON ((-110.05438 42.01103, -110.05436 42.0...",56,037,56_037
3139,56041,6.375806,"POLYGON ((-111.04663 41.14012, -111.04662 41.1...",56,041,56_041
3140,56043,6.194958,"POLYGON ((-108.55063 44.15179, -108.55056 44.1...",56,043,56_043


In [439]:
county_90th_percentile_with_geometry.dropna(inplace=True)

In [440]:
# Rank counties from highest to lowest based on PM25_90th values
county_ranked_epa = gdf_epa.sort_values(by='Design Value', ascending=False).reset_index(drop=True)
county_ranked_epa.reset_index(inplace=True)
county_ranked_epa['Design Value']

0      19.5
1      18.8
2      18.4
3      17.5
4      17.0
       ... 
518     3.3
519     3.2
520     2.3
521     2.2
522     2.0
Name: Design Value, Length: 523, dtype: float64

In [441]:
# Rank counties from highest to lowest based on PM25_90th values
county_ranked = county_90th_percentile_with_geometry.sort_values(by='PM25_90th', ascending=False).reset_index(drop=True)
county_ranked.reset_index(inplace=True)
county_ranked

,index,GEOID,PM25_90th,geometry,STATEFP,COUNTYFP,ID
0,0,06039,15.251729,"POLYGON ((-120.54554 37.02758, -120.54398 37.0...",06,039,06_039
1,1,06103,15.229901,"POLYGON ((-123.06543 40.28697, -123.06073 40.2...",06,103,06_103
2,2,06063,14.584223,"POLYGON ((-121.49703 40.43702, -121.49487 40.4...",06,063,06_063
3,3,06007,13.335710,"POLYGON ((-122.06874 39.84222, -122.06694 39.8...",06,007,06_007
4,4,06045,12.115083,"POLYGON ((-124.02325 40.00128, -123.93545 40.0...",06,045,06_045
...,...,...,...,...,...,...,...
518,518,23009,2.982917,"MULTIPOLYGON (((-68.03522 44.33274, -68.03375 ...",23,009,23_009
519,519,02170,2.819708,"POLYGON ((-153.00188 62.72583, -152.99961 62.7...",02,170,02_170
520,520,15001,2.157882,"POLYGON ((-156.06147 19.72813, -156.06076 19.7...",15,001,15_001
521,521,15009,1.892923,"MULTIPOLYGON (((-156.69742 20.91637, -156.6957...",15,009,15_009


In [442]:
np.nanmean(gdf_epa['Design Value'])

8.001338432122372

In [443]:
np.nanmean(county_90th_percentile_with_geometry['PM25_90th'])

7.2616725184684086

In [444]:
county_90th_percentile_with_geometry[county_90th_percentile_with_geometry['PM25_90th'] > 7.94]

,GEOID,PM25_90th,geometry,STATEFP,COUNTYFP,ID
1,01003,8.249587,"POLYGON ((-88.02858 30.22676, -88.02399 30.230...",01,003,01_003
36,01073,8.211253,"POLYGON ((-87.34095 33.47109, -87.33833 33.474...",01,073,01_073
48,01097,8.255512,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ...",01,097,01_097
76,02090,8.123247,"POLYGON ((-148.66326 64.59079, -148.64821 64.5...",02,090,02_090
102,04012,8.697489,"POLYGON ((-114.73122 33.30404, -114.7299 33.30...",04,012,04_012
...,...,...,...,...,...,...
2826,51013,8.130516,"POLYGON ((-77.16811 38.89652, -77.16636 38.897...",51,013,51_013
2958,53011,9.678374,"POLYGON ((-122.79596 45.82502, -122.7857 45.84...",53,011,53_011
3076,55059,7.943043,"POLYGON ((-88.30589 42.61082, -88.30252 42.610...",55,059,55_059
3087,55079,8.608215,"POLYGON ((-88.06976 42.86261, -88.06959 42.867...",55,079,55_079


In [445]:
gdf_epa[gdf_epa['Design Value'] > 7.94]

,State Name,County Name,State FIPS,County FIPS,Design Value,ID
3,Alabama,Etowah,01,055,8.2,01_055
4,Alabama,Jefferson,01,073,8.4,01_073
8,Alabama,Russell,01,113,9.1,01_113
11,Alaska,Fairbanks North Star,02,090,12.2,02_090
15,Arizona,Maricopa,04,013,10.5,04_013
...,...,...,...,...,...,...
506,Wisconsin,Eau Claire,55,035,8.1,55_035
508,Wisconsin,Grant,55,043,8.7,55_043
511,Wisconsin,Milwaukee,55,079,9.4,55_079
512,Wisconsin,Outagamie,55,087,8.0,55_087


In [446]:
import numpy as np
from scipy.stats import rankdata

# Merge the two DataFrames on 'GEOID' to get matched pairs for CDV and CDVE
# CDV (CDV) = Design Value from county_ranked_epa
# CDVE (calculated satellite value) = PM25_90th from county_ranked
merged_df = county_ranked.merge(county_ranked_epa[['ID', 'Design Value']], on='ID', how='inner')

# Drop rows with NaN values in either column, only keep counties with both values
merged_df.dropna(subset=['PM25_90th', 'Design Value'], inplace=True)

# Extract CDV and CDVE values for correlation calculations
cdv = merged_df['Design Value'].values  # EPA Design Values (CDV)
cdve = merged_df['PM25_90th'].values  # 90th percentile PM2.5 values (CDVE)

# Calculate Pearson correlation coefficient
cdv_mean = np.mean(cdv)
cdve_mean = np.mean(cdve)
numerator = np.sum((cdv - cdv_mean) * (cdve - cdve_mean))
denominator = np.sqrt(np.sum((cdv - cdv_mean)**2) * np.sum((cdve - cdve_mean)**2))
pearson_correlation = numerator / denominator

# Calculate Spearman rank correlation coefficient
cdv_ranks = rankdata(cdv, method='min')
cdve_ranks = rankdata(cdve, method='min')
d_i = cdv_ranks - cdve_ranks
n = len(cdv)
spearman_numerator = 6 * np.sum(d_i**2)
spearman_denominator = n * (n**2 - 1)
spearman_correlation = 1 - (spearman_numerator / spearman_denominator)

# Print the results
print("Pearson Correlation Coefficient (r):", pearson_correlation)
print("Spearman Rank Correlation Coefficient (r_s):", spearman_correlation)


Pearson Correlation Coefficient (r): 0.6612220031033197
Spearman Rank Correlation Coefficient (r_s): 0.6770502569274964


In [447]:
from scipy.stats import pearsonr, spearmanr

pearson_r, _ = pearsonr(cdv, cdve)
spearman_r, _ = spearmanr(cdv, cdve)

print("Pearson Correlation (scipy):", pearson_r)
print("Spearman Correlation (scipy):", spearman_r)

Pearson Correlation (scipy): 0.6612220031033196
Spearman Correlation (scipy): 0.679597176289752


In [405]:
merged_df

,index,GEOID,PM25_90th,geometry,STATEFP,COUNTYFP,ID,Design Value
0,0,06039,15.251729,"POLYGON ((-120.54554 37.02758, -120.54398 37.0...",06,039,06_039,13.2
1,1,06103,15.229901,"POLYGON ((-123.06543 40.28697, -123.06073 40.2...",06,103,06_103,9.9
2,2,06063,14.584223,"POLYGON ((-121.49703 40.43702, -121.49487 40.4...",06,063,06_063,17.0
3,3,06007,13.335710,"POLYGON ((-122.06874 39.84222, -122.06694 39.8...",06,007,06_007,11.6
4,4,06045,12.115083,"POLYGON ((-124.02325 40.00128, -123.93545 40.0...",06,045,06_045,11.1
...,...,...,...,...,...,...,...,...
518,518,23009,2.982917,"MULTIPOLYGON (((-68.03522 44.33274, -68.03375 ...",23,009,23_009,3.2
519,519,02170,2.819708,"POLYGON ((-153.00188 62.72583, -152.99961 62.7...",02,170,02_170,4.5
520,520,15001,2.157882,"POLYGON ((-156.06147 19.72813, -156.06076 19.7...",15,001,15_001,3.4
521,521,15009,1.892923,"MULTIPOLYGON (((-156.69742 20.91637, -156.6957...",15,009,15_009,4.0


In [ ]:
merged_df[merged_df['ID'] == '55_025']

,index,GEOID,PM25_90th,geometry,STATEFP,COUNTYFP,ID,Design Value
334,334,55025,8.233334,"POLYGON ((-89.83856 42.94994, -89.83854 42.955...",55,025,55_025,8.0


In [33]:
import pandas as pd

# Load your data into a DataFrame
# Assuming the DataFrame is already loaded and named `df`
gdf_epa.sort_values(by='ID', ascending=True, inplace=True)
county_90th_percentile_with_geometry.sort_values(by='ID', ascending=True, inplace=True)

# Drop rows with NaN in either the CDV or CDVE columns
filtered_df = gdf_epa.copy()
# Calculate the difference (bias for each row)
filtered_df['Difference'] = county_90th_percentile_with_geometry['PM25_90th'].values - gdf_epa['Design Value'].values

# Calculate the median bias instead of the mean
bias_median = filtered_df['Difference'].median()
print(filtered_df['Difference'].describe())
# Check for NaN values in the Difference column
print("NaN values in Difference:", filtered_df['Difference'].isna().sum())


# Calculate the bias range (min and max differences)
bias_min = filtered_df['Difference'].min()
bias_max = filtered_df['Difference'].max()

print(f"Bias (median): {bias_median:.2f} µg/m³")
print(f"Bias range: {bias_min:.2f} µg/m³ to {bias_max:.2f} µg/m³")


count    537.000000
mean       0.901490
std        2.059197
min       -7.733333
25%        0.033334
50%        0.633333
75%        1.266667
max       25.450000
Name: Difference, dtype: float64
NaN values in Difference: 0
Bias (median): 0.63 µg/m³
Bias range: -7.73 µg/m³ to 25.45 µg/m³


In [39]:
filtered_df['Difference'].isna().sum()

0